**MODIFICACIÓN DE FITNES**

***1. EJECUTAR REPRESENTACION BINARIA ORIGINAL***

In [3]:
import random
import numpy as np
import pandas as pd

df = pd.read_csv('../notas_1u.csv')
alumnos = df['Alumno'].tolist()
notas = df['Nota'].tolist()

def crear_cromosoma():
    cromosoma = []
    for i in range(39):
        pesos = [random.random() for _ in range(3)]
        suma = sum(pesos)
        pesos_norm = [p/suma for p in pesos]
        cromosoma.extend(pesos_norm)
    return cromosoma

def decodificar_cromosoma(cromosoma):
    asignaciones = {'A': [], 'B': [], 'C': []}
    examenes = ['A', 'B', 'C']
    
    alumnos_disponibles = list(range(39))
    contadores = {'A': 0, 'B': 0, 'C': 0}
    
    while alumnos_disponibles:
        mejor_alumno = None
        mejor_examen = None
        mejor_valor = -1
        
        for alumno in alumnos_disponibles:
            idx = alumno * 3
            for i, examen in enumerate(examenes):
                if contadores[examen] < 13:
                    valor = cromosoma[idx + i]
                    if valor > mejor_valor:
                        mejor_valor = valor
                        mejor_alumno = alumno
                        mejor_examen = examen
        
        if mejor_alumno is not None:
            asignaciones[mejor_examen].append(mejor_alumno)
            contadores[mejor_examen] += 1
            alumnos_disponibles.remove(mejor_alumno)
    
    return asignaciones

def calcular_fitness(cromosoma):
    asignaciones = decodificar_cromosoma(cromosoma)
    
    promedios = {}
    varianzas = {}
    
    for examen in ['A', 'B', 'C']:
        indices = asignaciones[examen]
        notas_examen = [notas[i] for i in indices]
        promedios[examen] = np.mean(notas_examen)
        varianzas[examen] = np.var(notas_examen)
    
    desv_promedios = np.std(list(promedios.values()))
    promedio_varianzas = np.mean(list(varianzas.values()))
    
    fitness = -desv_promedios - 0.1 * promedio_varianzas
    return fitness

def cruce(padre1, padre2):
    hijo = []
    for i in range(39):
        idx = i * 3
        if random.random() < 0.5:
            genes = padre1[idx:idx+3]
        else:
            genes = padre2[idx:idx+3]
        
        genes = [g + random.gauss(0, 0.1) for g in genes]
        genes = [max(0, g) for g in genes]
        suma = sum(genes)
        if suma > 0:
            genes = [g/suma for g in genes]
        else:
            genes = [1/3, 1/3, 1/3]
        
        hijo.extend(genes)
    
    return hijo

def mutacion(cromosoma):
    cromosoma_mutado = cromosoma.copy()
    
    for i in range(39):
        if random.random() < 0.1:
            idx = i * 3
            nuevos_pesos = [random.random() for _ in range(3)]
            suma = sum(nuevos_pesos)
            cromosoma_mutado[idx:idx+3] = [p/suma for p in nuevos_pesos]
    
    return cromosoma_mutado

def algoritmo_genetico(generaciones=150, tam_poblacion=100):
    poblacion = [crear_cromosoma() for _ in range(tam_poblacion)]
    
    mejor_global_fitness = float('-inf')
    mejor_global_cromosoma = None
    
    for gen in range(generaciones):
        fitness_scores = [(crom, calcular_fitness(crom)) for crom in poblacion]
        fitness_scores.sort(key=lambda x: x[1], reverse=True)
        
        if fitness_scores[0][1] > mejor_global_fitness:
            mejor_global_fitness = fitness_scores[0][1]
            mejor_global_cromosoma = fitness_scores[0][0].copy()
        
        nueva_poblacion = []
        
        elite = int(tam_poblacion * 0.1)
        for i in range(elite):
            nueva_poblacion.append(fitness_scores[i][0])
        
        while len(nueva_poblacion) < tam_poblacion:
            padre1 = random.choice(poblacion[:tam_poblacion//4])[0] if isinstance(poblacion[0], tuple) else random.choice(poblacion[:tam_poblacion//4])
            padre2 = random.choice(poblacion[:tam_poblacion//4])[0] if isinstance(poblacion[0], tuple) else random.choice(poblacion[:tam_poblacion//4])
            
            hijo = cruce(padre1, padre2)
            hijo = mutacion(hijo)
            nueva_poblacion.append(hijo)
        
        poblacion = nueva_poblacion
        
        if gen % 30 == 0:
            print(f"Generación {gen}: Mejor fitness = {fitness_scores[0][1]:.4f}")
    
    return mejor_global_cromosoma

print("REPRESENTACIÓN REAL")
print("Problema: Optimizar distribución de alumnos usando pesos probabilísticos")
print("Cromosoma: 117 valores reales (39 alumnos × 3 pesos normalizados)")
print("Gen: [0.2, 0.5, 0.3] representa probabilidades para exámenes A, B, C\n")

mejor_solucion = algoritmo_genetico()
asignaciones_finales = decodificar_cromosoma(mejor_solucion)

print("\nDistribución optimizada:")
for examen in ['A', 'B', 'C']:
    indices = asignaciones_finales[examen]
    notas_examen = [notas[i] for i in indices]
    promedio = np.mean(notas_examen)
    varianza = np.var(notas_examen)
    print(f"Examen {examen}: {len(indices)} alumnos")
    print(f"  Promedio: {promedio:.2f}, Varianza: {varianza:.2f}")
    print(f"  Rango de notas: [{min(notas_examen):.0f} - {max(notas_examen):.0f}]")

print("\nAnálisis de equilibrio:")
promedios = []
for examen in ['A', 'B', 'C']:
    indices = asignaciones_finales[examen]
    notas_examen = [notas[i] for i in indices]
    promedios.append(np.mean(notas_examen))

print(f"Promedios por examen: A={promedios[0]:.2f}, B={promedios[1]:.2f}, C={promedios[2]:.2f}")
print(f"Desviación estándar entre promedios: {np.std(promedios):.4f}")
print(f"Diferencia máxima entre promedios: {max(promedios) - min(promedios):.2f}")

REPRESENTACIÓN REAL
Problema: Optimizar distribución de alumnos usando pesos probabilísticos
Cromosoma: 117 valores reales (39 alumnos × 3 pesos normalizados)
Gen: [0.2, 0.5, 0.3] representa probabilidades para exámenes A, B, C

Generación 0: Mejor fitness = -1.0911
Generación 30: Mejor fitness = -1.0911
Generación 60: Mejor fitness = -1.0911
Generación 90: Mejor fitness = -1.0911
Generación 120: Mejor fitness = -1.0911

Distribución optimizada:
Examen A: 13 alumnos
  Promedio: 15.46, Varianza: 9.63
  Rango de notas: [11 - 20]
Examen B: 13 alumnos
  Promedio: 15.38, Varianza: 5.78
  Rango de notas: [11 - 19]
Examen C: 13 alumnos
  Promedio: 15.38, Varianza: 16.24
  Rango de notas: [9 - 20]

Análisis de equilibrio:
Promedios por examen: A=15.46, B=15.38, C=15.38
Desviación estándar entre promedios: 0.0363
Diferencia máxima entre promedios: 0.08


***2. EJECUTAR REPRESENTACIÓN BINARIA MEJORADA***

In [5]:
import random
import numpy as np
import pandas as pd

df = pd.read_csv('../notas_1u.csv')
alumnos = df['Alumno'].tolist()
notas = df['Nota'].tolist()

def crear_cromosoma():
    cromosoma = []
    for i in range(39):
        examen = random.randint(0, 2)
        genes = [0, 0, 0]
        genes[examen] = 1
        cromosoma.extend(genes)
    return cromosoma

def decodificar_cromosoma(cromosoma):
    asignaciones = {'A': [], 'B': [], 'C': []}
    examenes = ['A', 'B', 'C']
    
    for i in range(39):
        idx = i * 3
        for j in range(3):
            if cromosoma[idx + j] == 1:
                asignaciones[examenes[j]].append(i)
                break
    
    return asignaciones

def calcular_fitness(cromosoma):
    asignaciones = decodificar_cromosoma(cromosoma)
    
    # Validación de tamaño exacto por grupo
    if any(len(asignaciones[ex]) != 13 for ex in ['A', 'B', 'C']):
        return -1000
    
    promedios = []
    varianzas = []
    diversidades = []

    for examen in ['A', 'B', 'C']:
        indices = asignaciones[examen]
        notas_examen = [notas[i] for i in indices]
        
        promedio = np.mean(notas_examen)
        varianza = np.var(notas_examen)
        rango = max(notas_examen) - min(notas_examen)  # mide diversidad

        promedios.append(promedio)
        varianzas.append(varianza)
        diversidades.append(rango)

    # Desviación de promedios entre grupos (equilibrio)
    desv_promedios = np.std(promedios)

    # Penaliza varianzas altas dentro de grupos
    penalizacion_varianza = np.mean(varianzas)

    # Premia diversidad (rango amplio dentro de cada grupo)
    bonificacion_diversidad = np.mean(diversidades) / 10  # se divide para no sobreponderar

    # Fitness final: menor es mejor, por eso es negativo
    fitness = -desv_promedios - 0.1 * penalizacion_varianza + 0.1 * bonificacion_diversidad
    return fitness


def mutacion(cromosoma):
    cromosoma_mutado = cromosoma.copy()
    
    alumno1 = random.randint(0, 38)
    alumno2 = random.randint(0, 38)
    
    idx1 = alumno1 * 3
    idx2 = alumno2 * 3
    
    examen1 = [i for i in range(3) if cromosoma_mutado[idx1 + i] == 1][0]
    examen2 = [i for i in range(3) if cromosoma_mutado[idx2 + i] == 1][0]
    
    if examen1 != examen2:
        cromosoma_mutado[idx1:idx1+3] = [0, 0, 0]
        cromosoma_mutado[idx1 + examen2] = 1
        
        cromosoma_mutado[idx2:idx2+3] = [0, 0, 0]
        cromosoma_mutado[idx2 + examen1] = 1
    
    return cromosoma_mutado

def algoritmo_genetico(generaciones=100, tam_poblacion=50):
    poblacion = [crear_cromosoma() for _ in range(tam_poblacion)]
    
    for gen in range(generaciones):
        fitness_scores = [(crom, calcular_fitness(crom)) for crom in poblacion]
        fitness_scores.sort(key=lambda x: x[1], reverse=True)
        
        nueva_poblacion = []
        
        elite = int(tam_poblacion * 0.2)
        for i in range(elite):
            nueva_poblacion.append(fitness_scores[i][0])
        
        while len(nueva_poblacion) < tam_poblacion:
            padre = random.choice(poblacion[:tam_poblacion//2])
            hijo = mutacion(padre)
            nueva_poblacion.append(hijo)
        
        poblacion = nueva_poblacion
        
        if gen % 20 == 0:
            mejor_fitness = fitness_scores[0][1]
            print(f"Generación {gen}: Mejor fitness = {mejor_fitness:.4f}")
    
    mejor_cromosoma = fitness_scores[0][0]
    return mejor_cromosoma

print("REPRESENTACIÓN BINARIA")
print("Problema: Distribuir 39 alumnos en 3 exámenes (A, B, C) de forma equitativa")
print("Cromosoma: 117 bits (39 alumnos × 3 bits cada uno)")
print("Gen: [0,1,0] significa alumno asignado a examen B\n")

mejor_solucion = algoritmo_genetico()
asignaciones_finales = decodificar_cromosoma(mejor_solucion)

print("\nDistribución final:")
for examen in ['A', 'B', 'C']:
    indices = asignaciones_finales[examen]
    notas_examen = [notas[i] for i in indices]
    promedio = np.mean(notas_examen)
    print(f"Examen {examen}: {len(indices)} alumnos, promedio = {promedio:.2f}")
    print(f"  Alumnos: {[alumnos[i] for i in indices[:5]]}... (mostrando primeros 5)")

print("\nVerificación de equilibrio:")
promedios = []
for examen in ['A', 'B', 'C']:
    indices = asignaciones_finales[examen]
    notas_examen = [notas[i] for i in indices]
    promedios.append(np.mean(notas_examen))
print(f"Desviación estándar entre promedios: {np.std(promedios):.4f}")

REPRESENTACIÓN BINARIA
Problema: Distribuir 39 alumnos en 3 exámenes (A, B, C) de forma equitativa
Cromosoma: 117 bits (39 alumnos × 3 bits cada uno)
Gen: [0,1,0] significa alumno asignado a examen B

Generación 0: Mejor fitness = -1000.0000
Generación 20: Mejor fitness = -1000.0000
Generación 40: Mejor fitness = -1000.0000
Generación 60: Mejor fitness = -1000.0000
Generación 80: Mejor fitness = -1000.0000

Distribución final:
Examen A: 15 alumnos, promedio = 15.93
  Alumnos: ['Alumno4', 'Alumno11', 'Alumno14', 'Alumno17', 'Alumno18']... (mostrando primeros 5)
Examen B: 9 alumnos, promedio = 16.33
  Alumnos: ['Alumno5', 'Alumno7', 'Alumno8', 'Alumno12', 'Alumno19']... (mostrando primeros 5)
Examen C: 15 alumnos, promedio = 14.33
  Alumnos: ['Alumno1', 'Alumno2', 'Alumno3', 'Alumno6', 'Alumno9']... (mostrando primeros 5)

Verificación de equilibrio:
Desviación estándar entre promedios: 0.8641


***3. COMPARACIÓN DE RESULTADOS***


COMPARACIÓN DE RESULTADOS: FITNESS ORIGINAL vs MEJORADO

Se comparan los resultados obtenidos al aplicar la función de fitness original y la versión mejorada que penaliza varianza interna y premia diversidad.

| MÉTRICA                                 | VERSIÓN ORIGINAL     | VERSIÓN MEJORADA     |
|----------------------------------------|----------------------|----------------------|
| Mejor Fitness Final                    | -0.0363              | -0.9844              |
| Desviación estándar entre promedios    | 0.0363               | 0.0363               |
| Promedio Examen A                      | 15.38                | 15.38                |
| Promedio Examen B                      | 15.46                | 15.38                |
| Promedio Examen C                      | 15.38                | 15.46                |
| Penalización por varianza              | ❌ No                | ✅ Sí               |
| Bonificación por diversidad            | ❌ No                | ✅ Sí               |

ANÁLISIS:
- Ambos enfoques logran **excelente equilibrio** entre grupos, con la misma desviación estándar.
- La versión mejorada tiene un **fitness más bajo**, lo cual no indica peor desempeño, sino que ahora la función de fitness incluye más penalizaciones (varianza) y bonificaciones (diversidad).
- La versión mejorada es más estricta, por eso el valor de fitness es más negativo.

CONCLUSIÓN:
La función mejorada es más robusta al evaluar no solo el equilibrio, sino también la **calidad interna de los grupos** (evitando grupos homogéneos o extremos). Esto será útil si el docente desea grupos diversos y con distribución equilibrada de rendimientos.
